# IMPLEMENTACION DE VARIABLE ELIMINATION

## IMPLEMENTACIÓN CON SAMIAM

Se muestra imagenes de la implementación del modelo desarrollado en clase con la herramienta SAMIAM, con esta tambien comprararemos al momento de ver la implementación en python de Variable Elimination.

![img1](img1.png)

![img2](img2.png)

## IMPLEMENTACIÓN CON PYTHON

Para poder propagar de forma correcta las probabilidades en cada dependencia se requiere obtener cada nodo por su orden topologico para esto usamos la funcion GETORDER(), en la cual ademas se agrega las relaciones entre nodos, los valores asi como el estado de cada variable.

Exixten 3 estados 100, 50 y 0, los cuales representan i = 0, 1, 2.

In [9]:
def getOrder():
    return [
        {"node":"L", "parents":["G"],      "probabilities":[{"condition":{"G":100, "L":100}, "value":0.9},
                                                            {"condition":{"G":100, "L":0}, "value":0.1},
                                                            {"condition":{"G":0, "L":100}, "value":0.6},
                                                            {"condition":{"G":0, "L":0}, "value":0.4},
                                                            {"condition":{"G":50, "L":100}, "value":0.01},
                                                            {"condition":{"G":50, "L":0}, "value":0.99}]},

        {"node":"G", "parents":["D", "I"], "probabilities":[{"condition":{"G":100, "I":100, "D":100}, "value":0.5},
                                                            {"condition":{"G":100, "I":100, "D":0}, "value":0.9},
                                                            {"condition":{"G":100, "I":0, "D":100}, "value":0.05},
                                                            {"condition":{"G":100, "I":0, "D":0}, "value":0.3},
                                                            {"condition":{"G":0, "I":100, "D":100}, "value":0.3},
                                                            {"condition":{"G":0, "I":100, "D":0}, "value":0.08},
                                                            {"condition":{"G":0, "I":0, "D":100}, "value":0.25},
                                                            {"condition":{"G":0, "I":0, "D":0}, "value":0.3},
                                                            {"condition":{"G":50, "I":100, "D":100}, "value":0.2},
                                                            {"condition":{"G":50, "I":100, "D":0}, "value":0.02},
                                                            {"condition":{"G":50, "I":0, "D":100}, "value":0.7},
                                                            {"condition":{"G":50, "I":0, "D":0}, "value":0.3}]},

        {"node":"S", "parents":["I"],      "probabilities":[{"condition":{"I":100,"S":100}, "value":0.8},
                                                            {"condition":{"I":100,"S":0}, "value":0.2},
                                                            {"condition":{"I":0,"S":100}, "value":0.05},                                                                {"condition":{"I":0,"S":0}, "value":0.95}]}, 

        {"node":"I", "parents":[],         "probabilities":[{"condition":{"I":100}, "value":0.3},
                                                            {"condition":{"I":0}, "value":0.7}]},

        {"node":"D", "parents":[],         "probabilities":[{"condition":{"D":100}, "value":0.4},
                                                            {"condition":{"D":0}, "value":0.6}]}, 
    ]

Definimos algunas funciones de ayuda para el cálculo de las probabilidades.

In [2]:
def contains(cond, c):
    for key in c:
        if key in cond:
            if cond[key] != c[key]:
                return False
        else:
            return False
    return True

def getAllConditions(vars):
    pl = {
        "D":[100, 0],"I":[100, 0], "S":[100, 0], "G":[100, 50, 0], "L": [100, 0]
    }
    comb = []
    
    for v in vars:
        if len(comb) == 0:
            for t in pl[v]:
                comb.append({v:t})
        else:
            tmp = []
            for cm in comb:
                for t in pl[v]:
                    r = {v:t}
                    r.update(dict(cm))
                    tmp.append(r)
            comb = tmp
    return comb

def containsElim(old, cond):
    for key in cond["condition"]:
        if key in old["condition"]:
            if old["condition"][key] != cond["condition"][key]:
                return False
        else:
            return False
    return True



Esta función realiza la labor de eliminar las variables simplificando las expresions y sumando los valores.

In [ ]:
def eliminateSum(factor, oldCond, var):
    newP = []

    factor["variable"].remove(var)
    for cond in getAllConditions(factor["variable"]):
        newP.append({"condition": cond, "value":0.0})
    
    for cond in newP:
        for old in oldCond:
            if containsElim(old, cond):
                cond["value"] = cond["value"] + old["value"]

    return newP

Esta función obtiene los nuevos valores de las proporciones a travez de la multiplicacion (producto punto)

In [3]:
def join(factor1, factor2):
    newVars = list(factor1["variable"])
    newVars.extend([element for element in factor2["variable"] if element not in newVars])

    newP = []
    for cond in getAllConditions(newVars):
        prob = 1.0

        for c in factor2["p"]:
            if contains(cond, c["condition"]):
                prob *= c["value"]

        for c in factor1["p"]:
            if contains(cond, c["condition"]):
                prob *= c["value"]

        newP.append({'value': prob, 'condition': cond})

    return {"variable": newVars, "p": newP}

ADDFACTOR(), crea nuevos factores a cada iteración para a los cuales se aplica eliminación para obtener los nuevos valores de probabilidad.

In [4]:
def addFactor(v, evidence):

    factor = {"variable": list(v["parents"]), "p": v["probabilities"]}
    factor["variable"].append(v["node"])

    for e in evidence:
        if e in factor["variable"]:
            newP = []
            for c in factor["p"]:
                for cond in c["condition"]:
                    if cond == e and c["condition"][cond] == evidence[e]:
                        newP.append(c)
                        break
            factor["p"] = newP
            factor["p"] = eliminateSum(factor, factor["p"], e)

    return factor

Es la funcion principal, esta funcion recorre todos los nodos en orden topologico, y halla las nueva probabilidades deacuerdo a la evidencia presentada, estos son normalizados para la correcta visualización.

In [7]:
def ask(target, evidence):
    topologicalOrder = getOrder()

    factors = []

    for v in topologicalOrder:

        factors.append(addFactor(v, evidence))

        if target["node"] != v["node"] and not(v["node"] in evidence):
            temp = factors[0]

            for fact in factors[1:]:
                temp = join(temp, fact)

            temp["p"] = eliminateSum(temp, temp["p"], v["node"])

            factors = []
            factors.append(temp)

    result = factors[0]
    for fact in factors[1:]:
        result = join(result, fact)


    for ans in result["p"]:
        if target["node"] in ans["condition"]:
            if  target["value"] == ans["condition"][target["node"]]:
                return ans["value"]
    return "-1"

## Comparación de Resultados en Python con SamIam

El metodo a llamar es ASK, este contiene la condicion y en target


ask({"node":"L","value":100}, {"D":100})


target = {"node":"L","value":100}


condicion = {"D":100}


![img3](img3.png)

Como podemos observar en la imagen tenemos loS resultados de la evidencia D = 100, al cual se correcponde L = 100 con el valor 33.1%, que es el mismo obtenido por el algoritmo propuesto.

In [47]:
res = ask({"node":"L","value":100}, {"D":100})

print("Main Resp: " + str(res))

Main Resp: 0.331


De la misma forma si queremos la inferencia de L = 0 con la evidencia de D = 100, obtenemos 66.9% que se corresponde con el resultado obtenido.

In [48]:
res = ask({"node":"L","value":0}, {"D":100})

print("Main Resp: " + str(res))

Main Resp: 0.669


![img4](img4.png)

In [42]:
res = ask({"node":"G","value":100}, {"I":0})

print("Main Resp: " + str(res))

Main Resp: 0.2


In [43]:
res = ask({"node":"D","value":100}, {"I":0})

print("Main Resp: " + str(res))

Main Resp: 0.4
